In [1]:
%matplotlib inline
import pandas as pd
import pandasql as ps
from datetime import datetime
import seaborn
import matplotlib.pyplot as plt
import time

%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

In [2]:
pd.__version__

'1.0.3'

In [3]:
df = pd.DataFrame([[1, 2], [1, 5], [7, 8]],
     index=['A', 'B', 'C'],
     columns=['X', 'Y'])
df

,X,Y
A,1,2
B,1,5
C,7,8


In [4]:
df2 = pd.DataFrame([[1, 20], [4, 50], [70, 80]],
     index=['A', 'B', 'C'],
     columns=['X', 'Y'])
df2

,X,Y
A,1,20
B,4,50
C,70,80


In [5]:
def union_frames_sql(df,df2):
    simple_query = '''
        SELECT 
            X,
            Y
        FROM df
        UNION
        SELECT 
            X,
            Y
        FROM df2
        
        ORDER BY Y desc
        LIMIT 10
        '''
    return ps.sqldf(simple_query, locals())

In [6]:
start_time = time.time()
print (union_frames_sql(df,df2))
print("--- %s seconds ---" % (time.time() - start_time))


    X   Y
0  70  80
1   4  50
2   1  20
3   7   8
4   1   5
5   1   2
--- 0.017925024032592773 seconds ---


In [7]:
def union_frames_pandas(df,df2):
    df3=pd.concat([df,df2])
    return df3[['X', 'Y']].sort_values(by ='Y', ascending = False)[:10]

In [8]:
start_time = time.time()
print(union_frames_pandas(df,df2))
print("--- %s seconds ---" % (time.time() - start_time))

    X   Y
C  70  80
B   4  50
A   1  20
C   7   8
B   1   5
A   1   2
--- 0.003989458084106445 seconds ---


In [9]:
def join_frames_sql(df,df2):
    simple_query = '''
        SELECT 
            *
        FROM df
        JOIN
        df2
        ON
        df.X=df2.X
        ORDER BY Y desc
        LIMIT 10
        '''
    return ps.sqldf(simple_query, locals())

In [10]:
start_time = time.time()
print(join_frames_sql(df,df2))
print("--- %s seconds ---" % (time.time() - start_time))

   X  Y  X   Y
0  1  5  1  20
1  1  2  1  20
--- 0.009974241256713867 seconds ---


In [11]:
def join_frames_pandas(df,df2):
    df3 = pd.merge(df, df2, on='X', how='inner')
    return df3.sort_values(by ='Y_x', ascending = False)[:10]

In [12]:
start_time = time.time()
print(join_frames_pandas(df,df2))
print("--- %s seconds ---" % (time.time() - start_time))

   X  Y_x  Y_y
1  1    5   20
0  1    2   20
--- 0.0059833526611328125 seconds ---


In [13]:
def aggr_frames_sql(df,func):
    simple_query = '''
        SELECT 
            '''+func+'''(Y)
        FROM df
        GROUP BY X
        ORDER BY Y desc
        LIMIT 10
        '''
    return ps.sqldf(simple_query, locals())

In [15]:
start_time = time.time()
print(aggr_frames_sql(df,'avg'))
print("--- %s seconds ---" % (time.time() - start_time))

   avg(Y)
0     8.0
1     3.5
--- 0.008976221084594727 seconds ---


In [16]:
def avg_frame_pandas(df):
    return df.groupby('X').mean()

In [18]:
start_time = time.time()
print(avg_frame_pandas(df))
print("--- %s seconds ---" % (time.time() - start_time))

     Y
X     
1  3.5
7  8.0
--- 0.0039904117584228516 seconds ---
